In [1]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('simvision_pixel_TDC.csv', skiprows=1) ## Skip first row SimTime = 0

In [ ]:
### Original column names
### 'SimTime',
### 'ETROC2_tb.ETROC2_inst.PixelMatrix_HTree_inst.PixelCol_inst_1.Pixel_inst_2.PixelAnalog_inst.TDC_CAL[9:0]'
### 'ETROC2_tb.ETROC2_inst.PixelMatrix_HTree_inst.PixelCol_inst_1.Pixel_inst_2.PixelAnalog_inst.TDC_TOA[9:0]'
### 'ETROC2_tb.ETROC2_inst.PixelMatrix_HTree_inst.PixelCol_inst_1.Pixel_inst_2.PixelAnalog_inst.TDC_TOT[8:0]'
### 'ETROC2_tb.ETROC2_inst.PixelMatrix_HTree_inst.PixelCol1_inst_14.Pixel1_inst_14.PixelAnalog1_inst.TDC_CAL[9:0]'
### 'ETROC2_tb.ETROC2_inst.PixelMatrix_HTree_inst.PixelCol1_inst_14.Pixel1_inst_14.PixelAnalog1_inst.TDC_TOA[9:0]'
### 'ETROC2_tb.ETROC2_inst.PixelMatrix_HTree_inst.PixelCol1_inst_14.Pixel1_inst_14.PixelAnalog1_inst.TDC_TOT[8:0]'

### Replace column names (only when TDC file has been opened)
df.columns = ['SimTime', 'Row1Col2_TDC_CAL', 'Row1Col2_TDC_TOA', 'Row1Col2_TDC_TOT', 
              'Row14Col14_TDC_CAL', 'Row14Col14_TDC_TOA', 'Row14Col14_TDC_TOT']

In [ ]:
df.head()

In [ ]:
### Covert hex to decimal
row1col2_cal = df['Row1Col2_TDC_CAL'].squeeze()
row1col2_toa = df['Row1Col2_TDC_TOA'].squeeze()
b16 = lambda x: int(x, 16)
row1col2_cal = row1col2_cal.apply(b16)
row1col2_toa = row1col2_toa.apply(b16)

In [ ]:
### plotting
import matplotlib.pyplot as plt
%matplotlib inline

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
fig.suptitle("1st row, 2nd column pixel TDC values", fontsize=14)
ax1.hist(row1col2_cal)
ax1.set_xlabel('CAL code', fontsize=12)

ax2.scatter(row1col2_toa, df['Row1Col2_TDC_TOT'])
ax2.grid()
ax2.set_xlabel('TOA code', fontsize=12)
ax2.set_ylabel('TOT code', fontsize=12)

In [ ]:
### Covert hex to decimal
row14col14_cal = df['Row14Col14_TDC_CAL'].squeeze()
row14col14_toa = df['Row14Col14_TDC_TOA'].squeeze()
row14col14_cal = row14col14_cal.apply(b16)
row14col14_toa = row14col14_toa.apply(b16)

In [ ]:
### plotting
import matplotlib.pyplot as plt
%matplotlib inline

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
fig.suptitle("14th row, 14th column pixel TDC values", fontsize=14)
ax1.hist(row14col14_cal)
ax1.set_xlabel('CAL code', fontsize=12)

ax2.scatter(row14col14_toa, df['Row14Col14_TDC_TOT'])
ax2.grid()
ax2.set_xlim(425, 438)
ax2.set_xlabel('TOA code', fontsize=12)
ax2.set_ylabel('TOT code', fontsize=12)

In [ ]:
df2 = pd.read_csv('simv ision_fastcmd_data.csv', skiprows=1)

In [ ]:
### Original column names
### 'SimTime' 
### 'ETROC2_tb.FCGenLocal_inst.FastComByte[7:0]'

### Replace column names (only when fast command file has been opened)
df2.columns = ['SimTime', 'FastComByte']
df2.head()

In [ ]:
df2['FastComByte']
### F0 (1111 0000): Idle
### 69 (0110 1001): Charge injection
### 5A (0101 1010): BCR (Bunch counter reset at BC0, reset the counter to zero)
### 96 (1001 0110): L1A
### Reference: https://indico.cern.ch/event/1127562/contributions/4904781/attachments/2454504/4319592/ETROCemulator%20slides20220921.pdf

In [ ]:
### create a list of our conditions
conditions = [
    (df2['FastComByte'] == 'F0'),
    (df2['FastComByte'] == '69'),
    (df2['FastComByte'] == '5A'),
    (df2['FastComByte'] == '96')
    ]

### create a list of the values we want to assign for each condition
values = ['Idle', 'Charge injection', 'BCR', 'L1A']

### create a new column and use np.select to assign values to it using our lists as arguments
df2['fcmd_defn'] = np.select(conditions, values)

### display updated DataFrame
print(df2)

In [2]:
df3 = pd.read_csv('simvision.csv', skiprows=1)
df3.columns = ['SimTime', 'DOR', 'DOL']
df3 = df3.drop(columns=['DOL'])
df3 = df3.iloc[:-1 , :] ## drop the last row

In [3]:
sim = df3.diff(periods=-1).fillna(0)
sim['SimTime'] = sim['SimTime'].abs()*0.001
sim['nth'] = sim['SimTime']/1560.
sim.head(10)

,SimTime,DOR,nth
0,1560.0,1.0,1.0
1,1560.0,-1.0,1.0
2,3120.0,1.0,2.0
3,23400.0,-1.0,15.0
4,1560.0,1.0,1.0
5,3120.0,-1.0,2.0
6,4680.0,1.0,3.0
7,1560.0,-1.0,1.0
8,1560.0,1.0,1.0
9,4680.0,-1.0,3.0


In [4]:
df3['diff'] = sim['SimTime'] ## Time difference between (nth, (n+1)th) row, n = 0, 1, 2, ...
df3['cycle'] = sim['nth']
df3.head(10)

,SimTime,DOR,diff,cycle
0,133809921000,1,1560.0,1.0
1,133811481000,0,1560.0,1.0
2,133813041000,1,3120.0,2.0
3,133816161000,0,23400.0,15.0
4,133839561000,1,1560.0,1.0
5,133841121000,0,3120.0,2.0
6,133844241000,1,4680.0,3.0
7,133848921000,0,1560.0,1.0
8,133850481000,1,1560.0,1.0
9,133852041000,0,4680.0,3.0


In [13]:
### Repeat rows in a pandas DataFrame based on column value
### https://stackoverflow.com/questions/47336704/repeat-rows-in-a-pandas-dataframe-based-on-column-value
bit_df = df3.reindex(df3.index.repeat(df3.cycle))
bit_df = bit_df.drop(columns=['SimTime', 'diff', 'cycle'])
bit_df = bit_df.iloc[20:, :] ## drop the first 20 rows
bit_df = bit_df.iloc[:-2, :] ## drop the last 2 rows
bit_df = bit_df.reset_index(drop=True)
bit_df.head(25)

,DOR
0,0
1,0
2,1
3,1
4,1
5,0
6,1
7,0
8,0
9,0


In [14]:
bit_df2 = np.logical_xor(bit_df,1).astype(int) ## Make the another version with flipped bits

In [27]:
subset1 = []
subset2 = []

output = []
for i in range(int(bit_df.shape[0]/40)):
    x1, x2 = 0+40*i, 16+40*i
    y1, y2 = 16+40*i, 40+40*i
    bit16 = bit_df.iloc[range(x1, x2), :].values.flatten().tolist()
    bit24 = bit_df.iloc[range(y1, y2), :].values.flatten().tolist()
    s1 = ''.join([str(x) for x in bit16])
    s2 = ''.join([str(x) for x in bit24])
    output.append([s1, s2, hex(int(s1, 2))[2:], hex(int(s2, 2))[2:]])
    if i < 10:
        print(s1, s2, hex(int(s1, 2))[2:], hex(int(s2, 2))[2:])

new_df = pd.DataFrame(columns=['bit16', 'bit24', 'bit16_hex', 'bit24_hex'], data=output)

0011101000111100 100010110000000000000001 3a3c 8b0001
0011101000111100 011010110000000000000001 3a3c 6b0001
0011101000111100 100110110000000000000001 3a3c 9b0001
0011101000111100 110110110000000000000001 3a3c db0001
0011101000111100 011110110000000000000001 3a3c 7b0001
0011101000111100 000001110000000000000001 3a3c 70001
0011101000111100 110001110000000000000001 3a3c c70001
0011101000111100 101001110000000000000001 3a3c a70001
0011101000111100 000101110000000000000001 3a3c 170001
0011101000111100 010101110000000000000001 3a3c 570001


In [28]:
new_df.to_csv('result1.csv', index=False)

In [24]:
subset1 = []
subset2 = []

output = []
for i in range(int(bit_df.shape[0]/40)):
    x1, x2 = 0+40*i, 16+40*i
    y1, y2 = 16+40*i, 40+40*i
    bit16 = bit_df2.iloc[range(x1, x2), :].values.flatten().tolist()
    bit24 = bit_df2.iloc[range(y1, y2), :].values.flatten().tolist()
    s1 = ''.join([str(x) for x in bit16])
    s2 = ''.join([str(x) for x in bit24])
    output.append([s1, s2, hex(int(s1, 2))[2:], hex(int(s2, 2))[2:]])
    if i < 10:
        print(s1, s2, hex(int(s1, 2))[2:], hex(int(s2, 2))[2:])

new_df = pd.DataFrame(columns=['bit16', 'bit24', 'bit16_hex', 'bit24_hex'], data=output)

1100010111000011 011101001111111111111110 c5c3 74fffe
1100010111000011 100101001111111111111110 c5c3 94fffe
1100010111000011 011001001111111111111110 c5c3 64fffe
1100010111000011 001001001111111111111110 c5c3 24fffe
1100010111000011 100001001111111111111110 c5c3 84fffe
1100010111000011 111110001111111111111110 c5c3 f8fffe
1100010111000011 001110001111111111111110 c5c3 38fffe
1100010111000011 010110001111111111111110 c5c3 58fffe
1100010111000011 111010001111111111111110 c5c3 e8fffe
1100010111000011 101010001111111111111110 c5c3 a8fffe


In [26]:
new_df.to_csv('result2.csv', index=False)

In [ ]:
### 3C5C pattern recognition
### Only check whether the data has 3C5C pattern at anywhere or not
### 3 C 5 C (hex)
### 0011 1100 0101 1100 (binary)
pattern = [0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0]

matched = bit_df.rolling(len(pattern)).apply(lambda x: all(np.equal(x, pattern)), raw=True)
matched = matched.sum(axis = 1).astype(bool)   #Sum to perform boolean OR
idx_matched = np.where(matched)[0]
subset = [(match-len(pattern)+1, match+1) for match in idx_matched]
#subset = [range(match-len(pattern)+1, match+1) for match in idx_matched]   
#aligned_data = pd.concat([aligned_data.iloc[subs,:] for subs in subset], axis = 0)
#aligned_data.reset_index(inplace=True, drop=True)

In [ ]:
subset

In [ ]:
### try to implement Jinyuan's C code algorithm
bits = bit_df['DOR'].values.tolist() ## covert to python list

def FindHeaderPosition(nBits, ChkLength):
    pattern = [0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0] ## 3C5C, MSB is bit15
    nDiff = -1
    for i in range(0, 6000):
    #for i in range(1800000, 2000000):
        nDiff=0
        for k in range(ChkLength):
            for j in range(16):
                nA = nBits[i+j+k*40]
                nB = pattern[j]
                #print('ith bit, nA, nB', i, nA, nB)
                if(nA != nB): 
                    nDiff += 1
        if nDiff == 0:
            print(i)
            break

    nRt = -1
    if nDiff == 0:
        nRt=i+15
    else:
        nRt=0
    
    if nRt == -1:
        raise ValueError('Could not find any Header position')
    
    return nRt;

FindHeaderPosition(bits, 1)

#for a in range(10000):
#    res = FindHeaderPosition(bits, 4)
    #print(res)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.hist(df3['cycle'], bins=40, range=(0, 40), histtype='step')
plt.yscale('log')
plt.xticks(np.arange(0, 42, step=2))
plt.xlabel('# of clock cycles', fontsize=12)
#plt.grid(axis='x')

In [ ]:
### 3C5C pattern recognition
### Only check whether the data has 3C5C pattern at anywhere or not
### 3 C 5 C (hex)
### 0011 1100 0101 1100 (binary)
pattern = [0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0]

matched = df3.rolling(len(pattern)).apply(lambda x: all(np.equal(x, pattern)), raw=True)
matched = matched.sum(axis = 1).astype(bool)   #Sum to perform boolean OR
idx_matched = np.where(matched)[0]
subset = [range(match-len(pattern)+1, match+1) for match in idx_matched]

In [ ]:
### 6AF3 pattern
### 6 A F 3
### 0110 1010 1111 0011